In [1]:
from AlignmentMeter import *
%reload_ext autoreload
%autoreload 2
from datetime import datetime

**Test 1:** Really simple, just one alignment for the org and the leg

In [2]:
#oid, bid, hid, alignment, date, org_name
cols = ['oid', 'bid', 'hid', 'alignment', 'date', 'org_name']
data = [
        [-1, 'bill_A', 0, 'For', '2016-10-1', 'org_1'],
        [-1, 'bill_B', 0, 'Against', '2016-10-2', 'org_1']
       ]
org_alignments_df = pd.DataFrame(data, columns=cols)
org_alignments_df['date'] = org_alignments_df['date'].apply(datetime.strptime, args=['%Y-%m-%d'])
org_alignments_df

,oid,bid,hid,alignment,date,org_name
0,-1,bill_A,0,For,2016-10-01,org_1
1,-1,bill_B,0,Against,2016-10-02,org_1


In [3]:
cols = ['pid', 'result', 'bid', 'date']
data = [
        [1, 'For', 'bill_A', '2016-10-3'],
        [1, 'For', 'bill_B', '2016-10-3']
       ]
leg_votes_df = pd.DataFrame(data, columns=cols)
leg_votes_df['date'] = leg_votes_df['date'].apply(datetime.strptime, args=['%Y-%m-%d'])
leg_votes_df

,pid,result,bid,date
0,1,For,bill_A,2016-10-03
1,1,For,bill_B,2016-10-03


In [4]:
scores_df_lst = []
for ((oid, bid), oid_alignments_df) in org_alignments_df.groupby(['oid', 'bid']):
    if len(oid_alignments_df['alignment'].unique()) > 1:
        scores_df = handle_multi_alignment(oid_alignments_df, leg_votes_df)
        if len(scores_df.index):
            assert type(scores_df) == pd.DataFrame
            scores_df_lst.append(scores_df)
    else:
        scores_df = handle_single_alignment(oid_alignments_df, leg_votes_df)
        if len(scores_df.index):
            assert type(scores_df) == pd.DataFrame
            scores_df_lst.append(scores_df)
df = pd.concat(scores_df_lst)

In [5]:
df

,pid,total_votes,aligned_votes,alignment_percentage,bid,oid
0,1,1,1,1.0,bill_A,-1
0,1,1,0,0.0,bill_B,-1


In [6]:
assert df.loc[df.bid=='bill_A', 'total_votes'][0] == 1
assert df.loc[df.bid=='bill_A', 'aligned_votes'][0] == 1
assert df.loc[df.bid=='bill_A', 'alignment_percentage'][0] == 1

assert df.loc[df.bid=='bill_B', 'total_votes'][0] == 1
assert df.loc[df.bid=='bill_B', 'aligned_votes'][0] == 0
assert df.loc[df.bid=='bill_B', 'alignment_percentage'][0] == 0

**Test 2:** 1 distinct alignment for the organization, but they registered multiple alignments. Multiple votes for the leg, but all with the same alignment

In [7]:
#oid, bid, hid, alignment, date, org_name
cols = ['oid', 'bid', 'hid', 'alignment', 'date', 'org_name']
data = [
        [-1, 'bill_A', 0, 'For', '2016-10-1', 'org_1'],
        [-1, 'bill_A', 0, 'For', '2016-10-2', 'org_1'],
        [-1, 'bill_B', 0, 'Against', '2016-10-1', 'org_1'],
        [-1, 'bill_B', 0, 'Against', '2016-10-2', 'org_1']
       ]
org_alignments_df = pd.DataFrame(data, columns=cols)
org_alignments_df['date'] = org_alignments_df['date'].apply(datetime.strptime, args=['%Y-%m-%d'])
org_alignments_df

,oid,bid,hid,alignment,date,org_name
0,-1,bill_A,0,For,2016-10-01,org_1
1,-1,bill_A,0,For,2016-10-02,org_1
2,-1,bill_B,0,Against,2016-10-01,org_1
3,-1,bill_B,0,Against,2016-10-02,org_1


In [8]:
cols = ['pid', 'result', 'bid', 'date']
data = [
        [1, 'For', 'bill_A', '2016-10-3'],
        [1, 'For', 'bill_A', '2016-10-4'],
        [1, 'For', 'bill_B', '2016-10-3'],
        [1, 'For', 'bill_B', '2016-10-4'],
       ]
leg_votes_df = pd.DataFrame(data, columns=cols)
leg_votes_df['date'] = leg_votes_df['date'].apply(datetime.strptime, args=['%Y-%m-%d'])
leg_votes_df

,pid,result,bid,date
0,1,For,bill_A,2016-10-03
1,1,For,bill_A,2016-10-04
2,1,For,bill_B,2016-10-03
3,1,For,bill_B,2016-10-04


In [9]:
scores_df_lst = []
for ((oid, bid), oid_alignments_df) in org_alignments_df.groupby(['oid', 'bid']):
    if len(oid_alignments_df['alignment'].unique()) > 1:
        scores_df = handle_multi_alignment(oid_alignments_df, leg_votes_df)
        if len(scores_df.index):
            assert type(scores_df) == pd.DataFrame
            scores_df_lst.append(scores_df)
    else:
        scores_df = handle_single_alignment(oid_alignments_df, leg_votes_df)
        if len(scores_df.index):
            assert type(scores_df) == pd.DataFrame
            scores_df_lst.append(scores_df)
df = pd.concat(scores_df_lst)

In [10]:
df

,pid,total_votes,aligned_votes,alignment_percentage,bid,oid
0,1,1,1,1.0,bill_A,-1
0,1,1,0,0.0,bill_B,-1


In [11]:
assert df.loc[df.bid=='bill_A', 'total_votes'][0] == 1
assert df.loc[df.bid=='bill_A', 'aligned_votes'][0] == 1
assert df.loc[df.bid=='bill_A', 'alignment_percentage'][0] == 1

assert df.loc[df.bid=='bill_B', 'total_votes'][0] == 1
assert df.loc[df.bid=='bill_B', 'aligned_votes'][0] == 0
assert df.loc[df.bid=='bill_B', 'alignment_percentage'][0] == 0

**Test 3:** 1 distinct alignment for the organization, but they registered multiple alignments. Multiple votes for the leg with differing alignments. Bill B should not be in this output

In [12]:
#oid, bid, hid, alignment, date, org_name
cols = ['oid', 'bid', 'hid', 'alignment', 'date', 'org_name']
data = [
        [-1, 'bill_A', 0, 'For', '2016-10-1', 'org_1'],
        [-1, 'bill_A', 0, 'For', '2016-10-2', 'org_1'],
        [-1, 'bill_B', 0, 'Against', '2016-10-5', 'org_1'],
        [-1, 'bill_B', 0, 'Against', '2016-10-6', 'org_1']
       ]
org_alignments_df = pd.DataFrame(data, columns=cols)
org_alignments_df['date'] = org_alignments_df['date'].apply(datetime.strptime, args=['%Y-%m-%d'])
org_alignments_df

,oid,bid,hid,alignment,date,org_name
0,-1,bill_A,0,For,2016-10-01,org_1
1,-1,bill_A,0,For,2016-10-02,org_1
2,-1,bill_B,0,Against,2016-10-05,org_1
3,-1,bill_B,0,Against,2016-10-06,org_1


In [13]:
cols = ['pid', 'result', 'bid', 'date']
data = [
        [1, 'For', 'bill_A', '2016-10-3'],
        [1, 'Against', 'bill_A', '2016-10-4'],
        [1, 'For', 'bill_B', '2016-10-3'],
        [1, 'For', 'bill_B', '2016-10-4'],
       ]
leg_votes_df = pd.DataFrame(data, columns=cols)
leg_votes_df['date'] = leg_votes_df['date'].apply(datetime.strptime, args=['%Y-%m-%d'])
leg_votes_df

,pid,result,bid,date
0,1,For,bill_A,2016-10-03
1,1,Against,bill_A,2016-10-04
2,1,For,bill_B,2016-10-03
3,1,For,bill_B,2016-10-04


In [14]:
scores_df_lst = []
for ((oid, bid), oid_alignments_df) in org_alignments_df.groupby(['oid', 'bid']):
    if len(oid_alignments_df['alignment'].unique()) > 1:
        scores_df = handle_multi_alignment(oid_alignments_df, leg_votes_df)
        if len(scores_df.index):
            assert type(scores_df) == pd.DataFrame
            scores_df_lst.append(scores_df)
    else:
        scores_df = handle_single_alignment(oid_alignments_df, leg_votes_df)
        if len(scores_df.index):
            assert type(scores_df) == pd.DataFrame
            scores_df_lst.append(scores_df)
df = pd.concat(scores_df_lst)

In [15]:
df

,pid,total_votes,aligned_votes,alignment_percentage,bid,oid
0,1,1,0,0.0,bill_A,-1


In [16]:
assert df.loc[df.bid=='bill_A', 'total_votes'][0] == 1
assert df.loc[df.bid=='bill_A', 'aligned_votes'][0] == 0
assert df.loc[df.bid=='bill_A', 'alignment_percentage'][0] == 0

assert sum(df.bid=='bill_B') == 0

**Test 4:** 2 full strata for the org. Leg takes alignments during all strata

In [17]:
#oid, bid, hid, alignment, date, org_name
cols = ['oid', 'bid', 'hid', 'alignment', 'date', 'org_name']
data = [
        [-1, 'bill_A', 0, 'For', '2016-10-2', 'org_1'],
        [-1, 'bill_A', 0, 'Against', '2016-10-3', 'org_1'],
        [-1, 'bill_A', 0, 'For', '2016-10-4', 'org_1'],
        [-1, 'bill_A', 0, 'Against', '2016-10-5', 'org_1']
       ]
org_alignments_df = pd.DataFrame(data, columns=cols)
org_alignments_df['date'] = org_alignments_df['date'].apply(datetime.strptime, args=['%Y-%m-%d'])
org_alignments_df

,oid,bid,hid,alignment,date,org_name
0,-1,bill_A,0,For,2016-10-02,org_1
1,-1,bill_A,0,Against,2016-10-03,org_1
2,-1,bill_A,0,For,2016-10-04,org_1
3,-1,bill_A,0,Against,2016-10-05,org_1


In [18]:
cols = ['pid', 'result', 'bid', 'date']
data = [
        [1, 'For', 'bill_A', '2016-10-1'],
        [1, 'For', 'bill_A', '2016-10-2'],
        [1, 'For', 'bill_A', '2016-10-3'],
        [1, 'For', 'bill_A', '2016-10-4'],
        [1, 'For', 'bill_A', '2016-10-5'],
        [1, 'For', 'bill_A', '2016-10-6'],
    
        [2, 'Against', 'bill_A', '2016-10-1'],
        [2, 'Against', 'bill_A', '2016-10-2'],
        [2, 'Against', 'bill_A', '2016-10-3'],
        [2, 'Against', 'bill_A', '2016-10-4'],
        [2, 'Against', 'bill_A', '2016-10-5'],
        [2, 'For', 'bill_A', '2016-10-6']
       ]
leg_votes_df = pd.DataFrame(data, columns=cols)
leg_votes_df['date'] = leg_votes_df['date'].apply(datetime.strptime, args=['%Y-%m-%d'])
leg_votes_df

,pid,result,bid,date
0,1,For,bill_A,2016-10-01
1,1,For,bill_A,2016-10-02
2,1,For,bill_A,2016-10-03
3,1,For,bill_A,2016-10-04
4,1,For,bill_A,2016-10-05
5,1,For,bill_A,2016-10-06
6,2,Against,bill_A,2016-10-01
7,2,Against,bill_A,2016-10-02
8,2,Against,bill_A,2016-10-03
9,2,Against,bill_A,2016-10-04


In [19]:
scores_df_lst = []
for ((oid, bid), oid_alignments_df) in org_alignments_df.groupby(['oid', 'bid']):
    if len(oid_alignments_df['alignment'].unique()) > 1:
        scores_df = handle_multi_alignment(oid_alignments_df, leg_votes_df)
        if len(scores_df.index):
            assert type(scores_df) == pd.DataFrame
            scores_df_lst.append(scores_df)
    else:
        scores_df = handle_single_alignment(oid_alignments_df, leg_votes_df)
        if len(scores_df.index):
            assert type(scores_df) == pd.DataFrame
            scores_df_lst.append(scores_df)
df = pd.concat(scores_df_lst)

In [20]:
df

,aligned_votes,alignment_percentage,bid,oid,pid,total_votes
pid,,,,,,
1,2,0.50,bill_A,-1,1,4
2,1,0.25,bill_A,-1,2,4


In [21]:
assert df.loc[(df.bid=='bill_A') & (df.pid == 1), 'total_votes'].iloc[0] == 4
assert df.loc[(df.bid=='bill_A') & (df.pid == 1), 'aligned_votes'].iloc[0] == 2
assert df.loc[(df.bid=='bill_A') & (df.pid == 1), 'alignment_percentage'].iloc[0] == .5

assert df.loc[(df.bid=='bill_A') & (df.pid == 2), 'total_votes'].iloc[0] == 4
assert df.loc[(df.bid=='bill_A') & (df.pid == 2), 'aligned_votes'].iloc[0] == 1
assert df.loc[(df.bid=='bill_A') & (df.pid == 2), 'alignment_percentage'].iloc[0] == .25

**Test 5:** One strata vote, one single alignment

In [22]:
#oid, bid, hid, alignment, date, org_name
cols = ['oid', 'bid', 'hid', 'alignment', 'date', 'org_name']
data = [
        [-1, 'bill_A', 0, 'For', '2016-10-2', 'org_1'],
        [-1, 'bill_A', 0, 'Against', '2016-10-3', 'org_1'],
        [-1, 'bill_A', 0, 'For', '2016-10-4', 'org_1'],
        [-1, 'bill_A', 0, 'Against', '2016-10-5', 'org_1'],
    
        [-1, 'bill_B', 0, 'For', '2016-10-4', 'org_1']
       ]
org_alignments_df = pd.DataFrame(data, columns=cols)
org_alignments_df['date'] = org_alignments_df['date'].apply(datetime.strptime, args=['%Y-%m-%d'])
org_alignments_df

,oid,bid,hid,alignment,date,org_name
0,-1,bill_A,0,For,2016-10-02,org_1
1,-1,bill_A,0,Against,2016-10-03,org_1
2,-1,bill_A,0,For,2016-10-04,org_1
3,-1,bill_A,0,Against,2016-10-05,org_1
4,-1,bill_B,0,For,2016-10-04,org_1


In [23]:
cols = ['pid', 'result', 'bid', 'date']
data = [
        [1, 'For', 'bill_A', '2016-10-1'],
        [1, 'For', 'bill_A', '2016-10-2'],
        [1, 'For', 'bill_A', '2016-10-3'],
        [1, 'For', 'bill_A', '2016-10-4'],
        [1, 'For', 'bill_A', '2016-10-5'],
        [1, 'For', 'bill_A', '2016-10-6'],
    
        [1, 'Against', 'bill_B', '2016-10-5'],
       ]
leg_votes_df = pd.DataFrame(data, columns=cols)
leg_votes_df['date'] = leg_votes_df['date'].apply(datetime.strptime, args=['%Y-%m-%d'])
leg_votes_df

,pid,result,bid,date
0,1,For,bill_A,2016-10-01
1,1,For,bill_A,2016-10-02
2,1,For,bill_A,2016-10-03
3,1,For,bill_A,2016-10-04
4,1,For,bill_A,2016-10-05
5,1,For,bill_A,2016-10-06
6,1,Against,bill_B,2016-10-05


In [24]:
scores_df_lst = []
for ((oid, bid), oid_alignments_df) in org_alignments_df.groupby(['oid', 'bid']):
    if len(oid_alignments_df['alignment'].unique()) > 1:
        scores_df = handle_multi_alignment(oid_alignments_df, leg_votes_df)
        if len(scores_df.index):
            assert type(scores_df) == pd.DataFrame
            scores_df_lst.append(scores_df)
    else:
        scores_df = handle_single_alignment(oid_alignments_df, leg_votes_df)
        if len(scores_df.index):
            assert type(scores_df) == pd.DataFrame
            scores_df_lst.append(scores_df)
df = pd.concat(scores_df_lst)

In [25]:
df

,aligned_votes,alignment_percentage,bid,oid,pid,total_votes
pid,,,,,,
1,2,0.5,bill_A,-1,1,4
0,0,0.0,bill_B,-1,1,1


In [26]:
assert df.loc[(df.bid=='bill_A') & (df.pid == 1), 'total_votes'].iloc[0] == 4
assert df.loc[(df.bid=='bill_A') & (df.pid == 1), 'aligned_votes'].iloc[0] == 2
assert df.loc[(df.bid=='bill_A') & (df.pid == 1), 'alignment_percentage'].iloc[0] == .5

assert df.loc[(df.bid=='bill_B') & (df.pid == 1), 'total_votes'].iloc[0] == 1
assert df.loc[(df.bid=='bill_B') & (df.pid == 1), 'aligned_votes'].iloc[0] == 0
assert df.loc[(df.bid=='bill_B') & (df.pid == 1), 'alignment_percentage'].iloc[0] == 0